In [ ]:
import os
import glob

import emcee
import corner
import yaml
%pylab inline

import constants

In [ ]:
def corner_plot(survey_name, show_grid_fit=False, **kwargs):
    backend = emcee.backends.HDFBackend(os.path.join(constants.MCMC_DIR_BASE, survey_name, f'chain_{survey_name}.hdf5'), read_only=True)
    config = None
    for p in glob.glob('mcmc_cfg/*.yaml'):
        if p.split('/')[-1].casefold() == f'{survey_name.casefold()}.yaml':
            with open(p, 'r') as f:
                config = yaml.safe_load(f)
    if not config:
        raise RuntimeError
    tau = backend.get_autocorr_time(quiet=True)
    burnin = int(2 * np.max(tau))
    thin = int(0.5 * np.min(tau))
    # thin = int(np.max(tau))
    chain_len = len(backend.get_chain())
    samples = backend.get_chain(discard=burnin, flat=True, thin=thin)
    # log_prob_samples = backend.get_log_prob(discard=burnin, flat=True, thin=thin)
    
    # log_prior_samples = backend.get_blobs(discard=burnin, flat=True, thin=thin)
    # print(backend.get_blobs())
    
    #print(np.min(samples, axis=0), np.max(samples, axis=0))
    #print(f"Accepted steps: {backend.accepted}")
    print(f"Autocorrelation time: {tau}")
    print(f"Chain length: {chain_len}; Chain length / 50: {chain_len / 50:.2f}")
    print("Burn-in: {0}".format(burnin))
    print("Thin: {0}".format(thin))
    print("Flat chain shape: {0}; {1:.1f} autocorrelation times.".format(samples.shape, samples.shape[0] / np.max(tau)))
    # print("flat log prob shape: {0}".format(log_prob_samples.shape))
    # print("flat log prior shape: {0}".format(log_prior_samples.shape))
    if show_grid_fit:
        truths = [config['priors']['bias_QSO'],
                  config['priors']['beta_QSO'] if config['priors']['beta_QSO'] else config['priors']['bias_QSO']**-1,
                  config['priors']['par_sigma_smooth'],
                  config['priors']['drp_QSO'],
                  config['priors']['bias_hcd'],
                  config['priors']['beta_hcd']]
    else:
        truths = None
    
    corner.corner(samples, labels=[r'$b_{gal}$', 
                                   r'$\beta_{gal}$', 
                                   r'$\sigma_z$', r'$\delta_z$', r'$b_{DLA}$', r'$\beta_{DLA}$'], show_titles=True, 
                  truths=truths, truth_color='red',
                  **kwargs)
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'corner_{survey_name}.pdf'))
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'mcmc', f'corner_{survey_name}.png'))

In [ ]:
corner_plot('vuds')

In [ ]:
corner_plot('zDeep')

In [ ]:
corner_plot('mosdef')

In [ ]:
corner_plot('clamato')

In [ ]:
corner_plot('3dhst')